## 1. Initialize the Tree
   - Define a `Node` class to represent each node of the tree.
   - Define the `DecisionTree` class, with a `fit` function that initiates the process of building the tree using training data `X` and target labels `y`.

In [ ]:
class Node:
  def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
    self.feature = feature
    self.threshold = threshold
    self.left = left
    self.right = right
    self.value = value

  def is_leaf_node(self):
    return self.value is not None


class DecisionTree:
  def __init__(self):
    self.root = None

  def fit(self, X, y):
    pass

## 2. Grow the Tree Recursively (`_grow_tree` function)
   - For the current subset of `X` and `y`:
     - **Check Stopping Criteria**:
       - If all samples belong to one class, create a leaf node with this class as its value.
     - **Select Features**:
       - Select feature indices to consider for the split (in this implementation, all features are used).
     - **Find Best Split** (`_best_split` function):
       - For each feature:
         - Sort values of that feature.
         - Calculate potential split thresholds by finding midpoints between consecutive sorted values.
         - For each threshold, calculate the information gain and track the best one.
     - **Split Data**:
       - Split `X` and `y` into two subsets based on the best feature and threshold.
     - **Recursive Calls**:
       - Recursively grow left and right subtrees with the split data subsets.
     - **Return Node**:
       - Return a node with the selected feature and threshold, and attach the left and right subtrees.





In [ ]:
import numpy as np

class DecisionTree:
  def __init__(self):
    self.root = None

  def fit(self, X, y):
    self.root = self._build_tree(X, y)

  def _build_tree(self, X, y):
    n_samples, n_features = X.shape
    n_labels = len(np.unique(y))

    if(n_labels == 1):
      leaf_value = self._most_common_label(y)
      return Node(value=leaf_value)

    feat_idx = np.arange(n_features)

    best_feature, best_threshold = self._best_split(X, y, feat_idx)

    left_idxs , right_idxs = self._split(X[:, best_feature], best_threshold)
    left = self._build_tree(X[left_idxs, :], y[left_idxs])
    right = self._build_tree(X[right_idxs, :], y[right_idxs])
    return Node(best_feature, best_threshold, left, right)

  def _most_common_label(self, y):
    unique_labels, counts = np.unique(y, return_counts=True)
    return unique_labels[np.argmax(counts)]

  def _split(self, X_column, split_threshold):
    left_idxs = np.argwhere(X_column <= split_threshold).flatten()
    right_idxs = np.argwhere(X_column > split_threshold).flatten()
    return left_idxs, right_idxs

  def _best_split(self, X, y, feat_idxs):
    best_gain, split_idx, split_threshold = -1, None, None

    for feat_idx in feat_idxs:
      X_Column = X[:, feat_idx]
      X_Column_sorted = np.sort(X_Column)
      thresholds = (X_Column_sorted[:-1] + X_Column_sorted[1:])/2

      for threshold in thresholds:
        gain = self._information_gain(y, X_Column, threshold)

        if gain > best_gain:
          best_gain = gain
          split_idx = feat_idx
          split_threshold = threshold

    return split_idx, split_threshold


## 3. Evaluate Split Quality (`_information_gain` function)
   - **Split** (`_split` function):
     - Divide samples into left and right subsets based on the split threshold.
   - **Child Entropy**:
     - Compute weighted average of left and right child entropies based on their sizes.

In [ ]:
import numpy as np

class DecisionTree:
  def __init__(self):
    self.root = None

  def fit(self, X, y):
    self.root = self._build_tree(X, y)

  def _build_tree(self, X, y):
    n_samples, n_features = X.shape
    n_labels = len(np.unique(y))

    if(n_labels == 1):
      leaf_value = self._most_common_label(y)
      return Node(value=leaf_value)

    feat_idx = np.arange(n_features)

    best_feature, best_threshold = self._best_split(X, y, feat_idx)

    left_idxs , right_idxs = self._split(X[:, best_feature], best_threshold)
    left = self._build_tree(X[left_idxs, :], y[left_idxs])
    right = self._build_tree(X[right_idxs, :], y[right_idxs])
    return Node(best_feature, best_threshold, left, right)

  def _most_common_label(self, y):
    unique_labels, counts = np.unique(y, return_counts=True)
    return unique_labels[np.argmax(counts)]

  def _split(self, X_column, split_threshold):
    left_idxs = np.argwhere(X_column <= split_threshold).flatten()
    right_idxs = np.argwhere(X_column > split_threshold).flatten()
    return left_idxs, right_idxs

  def _best_split(self, X, y, feat_idxs):
    best_gain, split_idx, split_threshold = float('inf'), None, None

    for feat_idx in feat_idxs:
      X_Column = X[:, feat_idx]
      X_Column_sorted = np.sort(X_Column)
      thresholds = (X_Column_sorted[:-1] + X_Column_sorted[1:])/2

      for threshold in thresholds:
        gain = self._information_gain(y, X_Column, threshold)

        if gain < best_gain:
          best_gain = gain
          split_idx = feat_idx
          split_threshold = threshold

    return split_idx, split_threshold

  def _information_gain(self, y, X_column, threshold):
    # parent_entropy = self._entropy(y)

    left_idxs, right_idxs = self._split(X_column, threshold)

    if len(left_idxs) == 0 or len(right_idxs) == 0:
      return 0

    n, n_l, n_r = len(y), len(left_idxs), len(right_idxs)
    e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
    child_entropy = (n_l/n)*e_l + (n_r/n)*e_r
    # information_gain = parent_entropy - child_entropy
    information_gain = child_entropy
    return information_gain

  def _entropy(self, y):
    pass

## 4. Entropy Calculation (`_entropy` function)
   - Calculate the entropy for a subset `y`:
     - Compute probabilities for each class and use the entropy formula.

In [ ]:
import numpy as np

class DecisionTree:
  def __init__(self):
    self.root = None

  def fit(self, X, y):
    self.root = self._build_tree(X, y)

  def _build_tree(self, X, y):
    n_samples, n_features = X.shape
    n_labels = len(np.unique(y))

    if(n_labels == 1):
      leaf_value = self._most_common_label(y)
      return Node(value=leaf_value)

    feat_idx = np.arange(n_features)

    best_feature, best_threshold = self._best_split(X, y, feat_idx)

    left_idxs , right_idxs = self._split(X[:, best_feature], best_threshold)
    left = self._build_tree(X[left_idxs, :], y[left_idxs])
    right = self._build_tree(X[right_idxs, :], y[right_idxs])
    return Node(best_feature, best_threshold, left, right)

  def _most_common_label(self, y):
    unique_labels, counts = np.unique(y, return_counts=True)
    return unique_labels[np.argmax(counts)]

  def _split(self, X_column, split_threshold):
    left_idxs = np.argwhere(X_column <= split_threshold).flatten()
    right_idxs = np.argwhere(X_column > split_threshold).flatten()
    return left_idxs, right_idxs

  def _best_split(self, X, y, feat_idxs):
    best_gain, split_idx, split_threshold = -1, None, None

    for feat_idx in feat_idxs:
      X_Column = X[:, feat_idx]
      X_Column_sorted = np.sort(X_Column)
      thresholds = (X_Column_sorted[:-1] + X_Column_sorted[1:])/2

      for threshold in thresholds:
        gain = self._information_gain(y, X_Column, threshold)

        if gain > best_gain:
          best_gain = gain
          split_idx = feat_idx
          split_threshold = threshold

    return split_idx, split_threshold

  def _information_gain(self, y, X_column, threshold):
    # parent_entropy = self._entropy(y)

    left_idxs, right_idxs = self._split(X_column, threshold)

    if len(left_idxs) == 0 or len(right_idxs) == 0:
      return 0

    n, n_l, n_r = len(y), len(left_idxs), len(right_idxs)
    e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
    child_entropy = (n_l/n)*e_l + (n_r/n)*e_r
    # information_gain = parent_entropy - child_entropy
    information_gain = child_entropy
    return information_gain

  def _entropy(self, y):
    fid3 = np.mean(y)

    if fid3 == 0 or fid3 == 1:
      return 0
    else:
      return -fid3*np.log2(fid3) - (1-fid3)*np.log2(1-fid3)

## 5. Predict New Samples (`predict` function)
   - For each sample `x` in `X`:
     - **Traverse Tree** (`_traverse_tree` function):
       - Start at the root and navigate down:
         - Check if `x` satisfies the threshold condition at each node.
         - Continue left or right until reaching a leaf node.
       - **Return Prediction**:
         - Output the value (class) of the leaf node reached.

In [1]:
import numpy as np

class DecisionTree:
  def __init__(self):
    self.root = None

  def fit(self, X, y):
    self.root = self._build_tree(X, y)

  def _build_tree(self, X, y):
    n_samples, n_features = X.shape
    n_labels = len(np.unique(y))

    if(n_labels == 1):
      leaf_value = self._most_common_label(y)
      return Node(value=leaf_value)

    best_feature, best_threshold = self._best_split(X, y)

    left_idxs , right_idxs = self._split(X[:, best_feature], best_threshold)
    left = self._build_tree(X[left_idxs, :], y[left_idxs])
    right = self._build_tree(X[right_idxs, :], y[right_idxs])
    return Node(best_feature, best_threshold, left, right)

  def _most_common_label(self, y):
    unique_labels, counts = np.unique(y, return_counts=True)
    return unique_labels[np.argmax(counts)]

  def _split(self, X_column, split_threshold):
    left_idxs = np.argwhere(X_column <= split_threshold).flatten()
    right_idxs = np.argwhere(X_column > split_threshold).flatten()
    return left_idxs, right_idxs

  def _best_split(self, X, y):
    best_gain, split_idx, split_threshold = float("inf"), None, None

    for feat_idx in range(X.shape[1]):
      X_Column = X[:, feat_idx]
      X_Column_sorted = np.sort(X_Column)
      thresholds = (X_Column_sorted[:-1] + X_Column_sorted[1:])/2

      for threshold in thresholds:
        gain = self._information_gain(y, X_Column, threshold)

        if gain < best_gain:
          best_gain = gain
          split_idx = feat_idx
          split_threshold = threshold

    return split_idx, split_threshold

  def _information_gain(self, y, X_column, threshold):
    # parent_entropy = self._entropy(y)

    left_idxs, right_idxs = self._split(X_column, threshold)

    if len(left_idxs) == 0 or len(right_idxs) == 0:
      return 0

    n, n_l, n_r = len(y), len(left_idxs), len(right_idxs)
    e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
    child_entropy = (n_l/n)*e_l + (n_r/n)*e_r
    # information_gain = parent_entropy - child_entropy
    information_gain = child_entropy
    return information_gain

  def _entropy(self, y):
    fid3 = np.mean(y)

    if fid3 == 0 or fid3 == 1:
      return 0
    else:
      return -fid3*np.log(fid3) - (1-fid3)*np.log(1-fid3)

  def predict(self, X):
    predictions = np.array([self._traverse_tree(x, self.root) for x in X])
    return predictions

  def _traverse_tree(self, x, node):
    if node.is_leaf_node():
      return node.value

    if x[node.feature] <= node.threshold:
      return self._traverse_tree(x, node.left)
    return self._traverse_tree(x, node.right)

In [ ]:
X = np.array([
    [3, 7],
    [1, 8],
    [4, 5],
    [2, 6]
])

y = np.array([1, 0, 1, 0])

clf = DecisionTree()
clf.fit(X, y)
predictions = clf.predict(X)

def accuracy(y_test, y_pred):
  return np.mean(y_test == y_pred)

acc = accuracy(y, predictions)
print(acc)

1.0
